In this class, we will talk about model explainability but more in the context of data explainability or root cause analysis. In many cases building a very good machine learning model is not an ultimate goal. What is really wanted is the data understanding. A factory wants to know why the product is plagued with a defect, not to predict afterward if there is a defect or not. A football team wants to know which position is the best for scoring a goal, not what's the probability of scoring from a given position. And even when they want a prediction they would love to see the justification to trust the model. Often a nice plot is worth more than sophisticated machine-learning approaches.

In [ ]:
import dalex as dx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestRegressor

In [ ]:
data = load_wine()

In [ ]:
df = pd.DataFrame(data["data"], columns=data["feature_names"])
y = data["target"]
df["target"] = y

You should already be familiar with many data visualization techniques so we will not train it now. I just want to share a less popular type of data analysis. Usually plotting the target against any feature is not helpful but after some modification, we might be able to see some patterns.

In [ ]:
plt.plot(df.flavanoids, y, "bo")

For each value, we can plot the average target for data:
 - below that value
 - above that value
 - around that value

Please note that for the line "above that value" the more left we go the higher fraction of data is covered. The same with the "below that value"

In [ ]:
for col in df.columns.drop("target"):
    tmp = df.sort_values(col)
    plt.title(col)
    plt.plot(
        tmp[col],
        tmp[col].apply(lambda x: tmp[tmp[col] <= x].target.mean()),
        label="<=",
    )
    plt.plot(
        tmp[col],
        tmp[col].apply(lambda x: tmp[tmp[col] >= x].target.mean()),
        label=">=",
    )
    plt.plot(
        tmp[col],
        np.convolve(np.ones(20) / 20, tmp.target, mode="same"),
        label="~=~",
    )
    plt.legend()
    plt.grid()
    plt.show()

Ok, let's just train a model. We are not interested in top performance right now so we will skip hyperparameter optimization. Also, we want to find the pattern in the data we have, so we don't split the data into validation and test set.

In [ ]:
model = RandomForestRegressor()
x = df.drop("target", axis=1)
y = df.target
model.fit(x, y)

In [ ]:
plt.plot(df.target, model.predict(x), "bo")

Dalex is a python package for model explainability. We will use some of its functions to understand the data and the model better. First, we need to create an explainer model. Since we are not interested in checking the model performance but the relation between the data and the target we will use the whole dataset here. In the first case, we might want to use the testing set.

In [ ]:
exp = dx.Explainer(model, x, y)

In [ ]:
fi = exp.model_parts()

The first step will be feature importance. It's a basic analysis where we calculate the global impact of a feature. The idea in dalex default approach is to measure how much the model performance is worsening after removing this feature. Of course, it would require retraining the model, the optimal set of hyperparameters might be different and it might affect the results. To avoid these problems we do not retrain the model. Instead, we simulate its removal by assigning random values to it. To make it more realistic the values are not completely random, we just shuffle this column in a dataframe, do the prediction, check performance and repeat these steps multiple times.

In [ ]:
fi.plot()

Another useful tool is a partial dependency plot. For a given feature we observe what's the average output of our model for different values of this feature. For each considered value we set this value for each row in our dataframe and calculate an average prediction.

In [ ]:
exp.model_profile().plot()


We can also create similar plots for single rows. Here for each column, we present what would be the output from the model assuming we keep all remaining values and change the value of this one selected feature.

In [ ]:
exp.predict_profile(x.iloc[[15, 80]]).plot()

SHAP values are equivalents of Shapley values for the predictive models. It estimates the effect of a particular value of a particular feature for a prediction of a considered row. It's also done by replacing this value with proper sampling and replacing this value and measuring the effect on the prediction.

In [ ]:
exp.predict_parts(x.iloc[15], type="shap").plot()

In [ ]:
exp.predict_parts(x.iloc[15], type="shap").plot()

The result is based on sampling so the result for the same row can vary

In [ ]:
exp.predict_parts(x.iloc[88], type="shap").plot()

In [ ]:
exp.predict_parts(x.iloc[88], type="shap").result

**Task** For each class find the most representative examples and plot breakdown for them.

Imagine we have a model classifying dogs and cats. Then a good example would be to show e.g. 3 breeds of dogs and the same with cats. Showing 5 golden retrievers although cute is not the best approach.

There isn't a single best way how to approach this task. There are many good solutions. Think about what you want to achieve and then how to do it

In [ ]:
from typing import Any, Callable, Generator

from numpy.typing import ArrayLike


def add_column(df: pd.DataFrame, column: ArrayLike, name: str) -> pd.DataFrame:
    """Add column to the dataframe. Best works with `pd.DataFrame.pipe`
    method.

    Args:
        df: Dataframe to extend
        column: Column to add
        name: Name of the column

    Returns:
        Dataframe with added column
    """  # noqa: D205
    return pd.concat(
        [df.reset_index(), pd.DataFrame({name: column})], axis=1
    ).set_index("index")


def annotate_representatives(
    df: pd.DataFrame,
    cluster_strategy: Any,
    distance: Callable[[np.ndarray, np.ndarray], np.matrix],
) -> pd.DataFrame:
    """Divide into clusters and for each pick representative closest to
    the centroid.

    Args:
        df: Data for picking representatives from.
        cluster_strategy: Clustering algorithm.
        distance: Distance function.

    Returns:
        Dataframe with additional column: booleans signifying whether the
        record is representative.
    """  # noqa: D205
    # Cluster
    cluster_strategy = cluster_strategy.fit(df)
    centroids = cluster_strategy.cluster_centers_
    labels = cluster_strategy.labels_

    # Distance from each record to each centroid, stored in
    # (N_records, N_centroids) shape
    distances_mat = distance(df.to_numpy(), centroids)

    result = df.pipe(add_column, column=labels, name="cluster")
    result["repr"] = False
    ids = []

    # Can't "groupby" because the same records might occur => exclude them
    # after each iter

    # Iterate by columns (distances corresponding to a single centroid)
    for distances in distances_mat.T:
        ids.extend(
            result.pipe(add_column, column=distances, name="distance")
            .drop(ids)
            .sort_values("distance")
            .head(1)
            .index.tolist()
        )

    result.loc[pd.Index(ids), "repr"] = True
    return result.sort_values("index")

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import euclidean_distances

# Configurations
n_representatives = 5
cluster_strategy = KMeans(n_clusters=n_representatives)
distance_fn = euclidean_distances

# Annotate representatives for each class
result = pd.concat([
    annotate_representatives(cls_data, cluster_strategy, euclidean_distances)
    for cls, cls_data in df.groupby("target")
])

In [ ]:
import seaborn as sns
from sklearn.decomposition import PCA

sns.set_theme()

# Perform dim reduction and prepare data
plot_data = PCA().fit_transform(
    result.drop(["target", "cluster", "repr"], axis=1)
)
plot_data = pd.DataFrame(plot_data)
plot_data["target"] = result["target"]
plot_data["cluster"] = result["cluster"]
plot_data["repr"] = result["repr"]

# Plot for all classes
plt.tick_params(labelbottom=False, labelleft=False)
plt.title("All classes")
sns.scatterplot(data=plot_data, x=0, y=1, hue="target", palette="deep")
plt.show()

# Plot for each class
clss = result["target"].unique()
palette = sns.color_palette("deep")
for cls, color in zip(clss, palette):
    # Plotting params
    plt.title(f"Class {cls}")
    plt.tick_params(labelbottom=False, labelleft=False)

    # Plot points
    sns.scatterplot(data=plot_data, x=0, y=1, color="gray")
    sns.scatterplot(
        data=plot_data[plot_data["target"] == cls],
        x=0,
        y=1,
        color=color + (0.5,),
    )
    sns.scatterplot(
        data=plot_data[(plot_data["target"] == cls) & plot_data["repr"]],
        x=0,
        y=1,
        color="red",
    )

    plt.show()

There are other approaches that can be used for model explainability.
 - LIME - approximating model locally by a linear model
 - Anchor - approximating model locally by a rule-based model
 - Prototype - justifying a new prediction by showing a similar example from the data (a prototype)
 - Counterfactual Explanation - showing a similar example from the dataset with a different prediction to show what must be changed to change the prediction.

# Task

- take a dataset you want
- perform an exploratory data analysis (data visualization)
- create a sklearn pipeline for data preprocessing
- add new features (one hot encoding for example)
- add predictive model as the last step of the pipeline
- prepare a report with model explainability

Send it to gmiebs@cs.put.poznan.pl within 144 hours after the class is finished. Start the subject of the email with [IR]

Assume your report will be read by a domain expert from the area of the data, in our case a wine expert, without any computer science / data science skills. It means the person will not get much from raw plots and diagrams. Everything has to be explained to be understood.

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df.info()

In [ ]:
def all_vs_positive(df: pd.DataFrame) -> pd.DataFrame:
    loan_status_both = df.reset_index()
    loan_status_positive = df[df["loan_status"] == 1].reset_index()

    loan_status_both["loan_status"] = "all"
    loan_status_positive["loan_status"] = "positive"

    return pd.concat([loan_status_both, loan_status_positive])


def subplots(
    df_columns: int, n_cols: int, title: str
) -> tuple[plt.Figure, Generator[plt.Axes, str, None]]:
    n_rows = (df_columns + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4 * n_rows))
    fig.suptitle(title)

    def axes_gen() -> Generator[plt.Axes, str, None]:
        collapse_flag = False
        for ax in axes.ravel():
            if not collapse_flag:
                collapse_flag = (yield ax) == "collapse"
            else:
                ax.axis("off")
        yield

    return fig, axes_gen()

In [ ]:
kde_data = (
    (temp := all_vs_positive(df))
    .select_dtypes(include="number")
    .assign(loan_status=temp["loan_status"])
    .drop(["index", "id"], axis=1)
)

plotting_columns = list(filter(lambda x: x != "loan_status", kde_data.columns))
_, axes = subplots(
    df_columns=len(plotting_columns), n_cols=3, title="Numerical features"
)
for col, ax in zip(plotting_columns, axes):
    ax.tick_params(labelleft=False)
    sns.kdeplot(
        data=kde_data, x=col, hue="loan_status", ax=ax, fill=True, bw_adjust=2
    )
axes.send("collapse")
plt.tight_layout()
plt.show()

In [ ]:
hist_data = (
    (temp := all_vs_positive(df))
    .select_dtypes(include="object")
    .assign(loan_status=temp["loan_status"])
    .assign(loan_intent=temp["loan_intent"].str[:3] + ".")
)

plotting_columns = list(
    filter(lambda x: x != "loan_status", hist_data.columns)
)
fig, axes = subplots(
    df_columns=len(plotting_columns), n_cols=2, title="Categorical features"
)
for col, ax in zip(plotting_columns, axes):
    total_value_counts = hist_data[hist_data["loan_status"] == "all"][
        col
    ].value_counts()
    positive_value_counts = hist_data[hist_data["loan_status"] == "positive"][
        col
    ].value_counts()

    positive_value_counts = (
        (positive_value_counts / total_value_counts * 100)
        .astype(int)
        .reset_index()
    )
    total_value_counts = total_value_counts.reset_index().assign(count=100)

    if col != "loan_grade":
        order = positive_value_counts.sort_values("count")[col]
    else:
        order = positive_value_counts[col]

    sns.barplot(
        data=total_value_counts,
        y=col,
        x="count",
        ax=ax,
        color="gray",
        order=order,
    )
    sns.barplot(
        data=positive_value_counts, y=col, x="count", ax=ax, order=order
    )
axes.send("collapse")

fig.set_figwidth(20)
plt.tight_layout()
plt.show()